In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
base_dir='/kaggle/input/acea-water-prediction'

In [ ]:
os.listdir(base_dir)

In [ ]:
aq_d=pd.read_csv(os.path.join(base_dir,'Aquifer_Doganella.csv'))
aq_d.head()

In [ ]:
def find_missing_values(df):
    missing_val_tbl=pd.DataFrame(df.isnull().sum(),columns=['Count'])    
    return missing_val_tbl

In [ ]:
def display_missing_val(missing_val_tbl, title):
    
    plt.figure(figsize=(15,15))
    missing_val_tbl.sort_values(by='Count',ascending=True).plot(kind='barh')
    plt.title(title)
    plt.show()

In [ ]:
display_missing_val(find_missing_values(aq_d),'Missing values for Aquifer_Doganella')

In [ ]:
aq_a=pd.read_csv(os.path.join(base_dir,'Aquifer_Auser.csv'))
aq_a.head()

In [ ]:
display_missing_val(find_missing_values(aq_a),'Missing values for Aquifer_Auser')

In [ ]:
aq_l=pd.read_csv(os.path.join(base_dir,'Aquifer_Luco.csv'))
aq_l.head()

In [ ]:
display_missing_val(find_missing_values(aq_l),'Missing values for Aquifer_Luco')

In [ ]:
aq_p=pd.read_csv(os.path.join(base_dir,'Aquifer_Petrignano.csv'))
aq_p.head()

In [ ]:
display_missing_val(find_missing_values(aq_p),'Missing values for Aquifer_Petrignano')

# Water Spring

In [ ]:
ws_a=pd.read_csv(os.path.join(base_dir,'Water_Spring_Amiata.csv'))
ws_a.head()

In [ ]:
display_missing_val(find_missing_values(ws_a),'Missing values for Water_Spring_Amiata')

In [ ]:
ws_m=pd.read_csv(os.path.join(base_dir,'Water_Spring_Madonna_di_Canneto.csv'))
ws_m.head()

In [ ]:
display_missing_val(find_missing_values(ws_m),'Missing values for Water_Spring_Madonna_di_Canneto')

In [ ]:
ws_l=pd.read_csv(os.path.join(base_dir,'Water_Spring_Lupa.csv'))
ws_l.head()

In [ ]:
display_missing_val(find_missing_values(ws_m),'Missing values for Water_Spring_Lupa')

In [ ]:
all_ws={'Amiata':ws_a,'Madonna_di_Canneto':ws_m,'Lupa':ws_l}

In [ ]:
def analyse_heatmap(src,title='', thres=.6):
    print(f'Heatmap for {title}')
    fig, ax=plt.subplots(1,1,figsize=(15,12))

    corr=src.corr()
    mask=np.tril(np.ones_like(corr, dtype=bool))
    mask[(abs(corr)<thres) & (mask==False)]=True
    sns.heatmap(corr,mask=mask,ax=ax,vmin=-1, vmax=1, annot=True)
    ax.hlines(range(0, src.shape[1]),*ax.get_xlim(), lw=1)
    ax.vlines(range(0, src.shape[1]), *ax.get_xlim(), lw=1)
    plt.show()

In [ ]:
analyse_heatmap(ws_a, 'Amiata' )

In [ ]:
analyse_heatmap(ws_m,'Madonna_di_Canneto',0)

In [ ]:
print(aq_d.info())

In [ ]:
aq_d.shape

In [ ]:
aq_d.columns

# Rivers

In [ ]:
river_a=pd.read_csv(os.path.join(base_dir,'River_Arno.csv'))
river_a.head()

In [ ]:
display_missing_val(find_missing_values(river_a),'Missing values for River_Arno')

In [ ]:
fig, ax=plt.subplots(1,1,figsize=(15,12))

corr=river_a.corr()
mask=np.tril(np.ones_like(corr, dtype=bool))
mask[(abs(corr)<.6) & (mask==False)]=True
sns.heatmap(corr,mask=mask,ax=ax,vmin=-1, vmax=1, annot=True)
ax.hlines(range(0, river_a.shape[1]),*ax.get_xlim(), lw=1)
ax.vlines(range(0, river_a.shape[1]), *ax.get_xlim(), lw=1)
plt.show()

In [ ]:
corr_pos=corr.abs()
hi_corr_val=np.where(corr_pos>.6)
hi_corr_val= [ (corr_pos.columns[x], corr_pos.columns[y]) for x, y in zip(*hi_corr_val) if x!=y and x<y]

There is correlation between the rainfall in different areas

In [ ]:
# plotting correlated features
river_a.index=river_a.Date
df=river_a.loc['01/01/2020':'01/05/2020']
fig, ax=plt.subplots(5,3, figsize=(20,20))
ax=ax.ravel()
for i in range(len(hi_corr_val[:15])):    
    ax[i]=df[[hi_corr_val[i][0],hi_corr_val[i][1],'Date']].plot(x='Date', ax=ax[i])
    ax[i].set_title(hi_corr_val[i][0]+' vs '+hi_corr_val[i][1])
plt.subplots_adjust(wspace=.3, hspace=.3)
plt.show()


In [ ]:
river_a.index=river_a.Date
river_a=river_a.drop(columns=['Date'], axis=1)

In [ ]:

riv_cols=river_a.columns
fig , ax= plt.subplots(4,len(riv_cols)//4,figsize=(30,25))
axes=ax.ravel()

for  i in range(len(river_a.columns)):    
    axes[i]=river_a[riv_cols[i]].plot(ax=axes[i])
    axes[i].set_title(riv_cols[i])
plt.subplots_adjust(wspace=.3, hspace=.3)
plt.show()

In [ ]:
river_a.Hydrometry_Nave_di_Rosano.rolling(window=12).mean().plot(figsize=(10,8))

In [ ]:
df_r=river_a.Hydrometry_Nave_di_Rosano

In [ ]:
train=df_r.loc['01/01/2020':'30/05/2020']
test=df_r.loc['01/06/2020':]

In [ ]:
train=train.dropna()

In [ ]:
# Fit an AR model
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.ar_model import AR, ARResults

In [ ]:
model=AR(train)


In [ ]:
ARfit=model.fit(method='mle')
print(f'Lag: {ARfit.k_ar}')
print(f'Coeff: {ARfit.params}')

In [ ]:
start=len(train)
end=len(train)+len(test)-1
preds=ARfit.predict(start=start, end=end, dynamic=False).rename('AR_Preds_River_Level')

In [ ]:
train.plot(legend=True)
preds.plot(legend=True, figsize=(10,8))

In [ ]:
# Evaluate the model
from sklearn.metrics import mean_squared_error
print(f'MSE:{mean_squared_error(test, preds)}')

In [ ]:
# Forecast using stat models
df_r=df_r.dropna()
#df_r.index.freq='MS'
model=AR(df_r)
ARfit=model.fit(maxlag=13, model='mle')
fcast=ARfit.predict(start=len(df_r), end=len(df_r)+480, dynamic=False).rename('Forecast')
df_r.plot(legend=True)
fcast.plot(legend=True, figsize=(10,8));

 # Test for stationarity
A time series is stationary if the mean and variance are fixed between two equidistant points.

Augmented Dickey-Fuller test: A small p-value rejects the null hypothesis which assumes the data is non stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller,grangercausalitytests
from statsmodels.tools.eval_measures import mse, rmse, meanabs

In [ ]:
def adf_test(series, title=''):
    print(f'Augmented Dickey-Fuller Test:{title}')
    adf_series=adfuller(series.dropna(), autolag='AIC')
    labels=['ADF Test Statistic', 'p-value','lags_used', '# of observations']
    adf_series_result=pd.Series(adf_series[0:4],index=labels)
    for k,v in adf_series[4].items():
        adf_series_result[f'critical value: ({k})']=v
    print(adf_series_result)
    if adf_series_result[1]<.05:
        print("Strong evidence against null hypothesis; Data is stationary")
    else:
        print("Weak evidence against null hypothesis; Data is non-stationary")

In [ ]:
adf_test(df_r, 'Rivers')

In this case the p-value is very low and we reject the null hypothesis and the data is stationary.

# Granger Causality Tests

In [ ]:
gct_df=river_a.copy()
gct_df.index=pd.to_datetime(gct_df.index)
cols=list(gct_df.columns)
cols.remove('Hydrometry_Nave_di_Rosano')
target='Hydrometry_Nave_di_Rosano'
for col in cols:
    gct_df[[col,target]].plot(figsize=(10,8))


In [ ]:
for col in cols:
    print(f'Granger Causality Tests for {col}')
    grangercausalitytests(gct_df[[col,target]].dropna(), maxlag=3)
    print()

We are looking at extremely low p-values.

# Revealing seasonality with month plots and quarter plots

In [ ]:
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
df_r.index=pd.to_datetime(df_r.index)

In [ ]:
# Resample to create monthly data
m_dfr=df_r.resample(rule='M').mean()
month_plot(m_dfr);

In [ ]:
# Resample to create quarterly data
q_dfr=df_r.resample(rule='Q').mean()
quarter_plot(q_dfr);

# Lake

In [ ]:
lake_b=pd.read_csv(os.path.join(base_dir,'Lake_Bilancino.csv'))
lake_b.head()

In [ ]:
display_missing_val(find_missing_values(river_a),'Missing values for Lake_Bilancino')

In [ ]:
lake_b.index=lake_b.Date
#lake_b.drop(columns=['Date'], axis=1, inplace=True)

In [ ]:
analyse_heatmap(lake_b, 'Bilancino',.6)

In [ ]:
def get_correlated_columns(corr):
    corr_pos=corr.abs()
    hi_corr_val=np.where(corr_pos>.6)
    hi_corr_val= [ (corr_pos.columns[x], corr_pos.columns[y]) for x, y in zip(*hi_corr_val) if x!=y and x<y]
    return hi_corr_val

In [ ]:
lake_hicorrvals=get_correlated_columns(lake_b.corr())
print(lake_hicorrvals)

In [ ]:
# plotting correlated features
def plot_correlated_features(wb,hi_corr_val, title=''):
    print(f'plot correlated features for {title}')
    wb.index=wb.Date
    df=wb.loc['01/01/2020':'01/05/2020']
    fig, ax=plt.subplots(5,len(hi_corr_val)//5, figsize=(20,20))
    ax=ax.ravel()
    for i in range(len(hi_corr_val[:15])):    
        ax[i]=df[[hi_corr_val[i][0],hi_corr_val[i][1],'Date']].plot(x='Date', ax=ax[i])
        ax[i].set_title(hi_corr_val[i][0]+' vs '+hi_corr_val[i][1])
    plt.subplots_adjust(wspace=.3, hspace=.3)
    plt.show()


In [ ]:
plot_correlated_features(lake_b,lake_hicorrvals, 'Lake')

# WIP

In [ ]:
adf_test(lake_b.Lake_Level, 'Lake')

In [ ]:
adf_test(lake_b.Flow_Rate, 'Lake')

In [ ]:
lake_b.index=pd.to_datetime(lake_b.index)

In [ ]:
# Resample to create monthly data
m_lk_lvl=lake_b.Lake_Level.resample(rule='M').mean()
month_plot(m_lk_lvl);

In [ ]:
# Resample to create quarterly data
q_lk_lvl=lake_b.Lake_Level.resample(rule='Q').mean()
quarter_plot(q_lk_lvl);

In [ ]:
! pip install pmdarima

In [ ]:
from pmdarima import auto_arima

In [ ]:
auto_arima(lake_b.Lake_Level.dropna(),error_action='ignore').summary()